# アカウントのテキストと画像をもらう

In [ ]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token)
        self.count = 5


    @property
    def text_zget(self):
        """_summary_
        usernameにアカウント名を入力

        画像を取る場合は以下を有効にする
        for tweets_img in target_tweets.includes['media']:

        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []

        target_response = self.client.get_user(username='hikarinspecial') # usernameにアカウント名を入力
        target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude='retweets')

        while True:
            print(target_tweets)
            next_token = target_tweets.meta['next_token']

            # テキストを取る
            for tweets in target_tweets.data:
                text = tweets.data['text']
                print(text)

            # 画像を取る
            for tweets_img in target_tweets.includes['media']:
                img_url = tweets_img['url']
                print(img_url)

            tweet['tweet'].append({"text": text,"img": img_url})
            print(tweet)

            with open('imada_mio.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)

            target_tweets = self.client.get_users_tweets(target_response.data['id'], expansions=["attachments.media_keys"], max_results=10, media_fields=['preview_image_url', 'url'], exclude='retweets', pagination_token=next_token)

            if next_token is None:
                break



t = Tweet_text()
t.text_get


        # for tweets in tweepy.Paginator(self.client.get_users_tweets, target_response.data['id'], max_results=10, media_fields=['preview_image_url', 'url'], expansions=['attachments.media_keys'], tweet_fields=["entities"]).flatten(limit=250):


# v1でテキストと画像取得できた

- v2 テキストと画像取得しようとしたけどうまくいかない→ v1.1でやった方が楽という結論
- v1でOK

In [71]:
import tweepy
import API_config_katudon
import json
import requests
import re
import os


class Tweet_text:

    def __init__(self, username):

        auth = tweepy.OAuthHandler(API_config_katudon.API_KEY, API_config_katudon.API_SECRET)
        auth.set_access_token(API_config_katudon.ACCESS_TOKEN, API_config_katudon.ACCESS_TOKEN_SECRET)
        self.api = tweepy.API(auth, wait_on_rate_limit=True)
        self.__username = username

    def text_image_get(self):


        count_no = 200
        tweet: dict[dict] = {}
        tweet['tweet'] = []

        results: iter = tweepy.Cursor(self.api.user_timeline, # タイムラインの取得
                    id=self.__username, # 取得対象のユーザーを指定
                    include_entities=True, # 省略されたリンクを全て取得
                    tweet_mode='extended', # 省略されたツイートを全て取得
                    lang='ja',
                    exclude_replies=False,
                    include_rts=False).items(limit=count_no) # 取得件数を指定

        index = 1

        for result in results:
            if 'media' in result.entities:
                image_url = result.extended_entities['media'][0]['media_url']
                texts = result.full_text
                text = texts.split()
                tweet_text = text[0]

                print(tweet)

                img_response = requests.get(image_url)
                match img_response.status_code:
                    case 200:
                        os.makedirs(f'{self.__username}', mode=0o777, exist_ok=True)
                        with open(f'{self.__username}/{index}.jpg', 'wb') as image:
                            image.write(img_response.content)

                        tweet['tweet'].append({"text": tweet_text, 'img': image_url, 'img_file': f'{index}.jpg'})
                        index += 1

                with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                    json.dump(tweet, f, indent=4, ensure_ascii=False)

In [72]:
t = Tweet_text('erika_1837')
t.text_image_get()

Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang
Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


{'tweet': []}
{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}]}
{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}]}
{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}, {'text': 'このブラ盛れるっ😋✨', 'img': 'http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg', 'img_file': '3.jpg'}]}
{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}, {'text': 'このブラ盛れるっ😋✨', 'img': 'http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg'

Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}, {'text': 'このブラ盛れるっ😋✨', 'img': 'http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg', 'img_file': '3.jpg'}, {'text': 'おっぱいようでございます🤗', 'img': 'http://pbs.twimg.com/media/FQGLoR6akAIGbYI.jpg', 'img_file': '4.jpg'}, {'text': '見えちゃいまひた😆', 'img': 'http://pbs.twimg.com/media/FPgK0ScaAAQ2gsU.jpg', 'img_file': '5.jpg'}, {'text': 'みんないつもコメントありがとぉ〜🥰', 'img': 'http://pbs.twimg.com/media/FOSOdnWacAcyjpr.jpg', 'img_file': '6.jpg'}]}
{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}, {'text': 'このブラ盛れるっ😋✨', 'img': 'http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg', 'img_file': '3.jpg'}, {'text': 'おっぱいようでございます

Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


...

{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}, {'text': 'このブラ盛れるっ😋✨', 'img': 'http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg', 'img_file': '3.jpg'}, {'text': 'おっぱいようでございます🤗', 'img': 'http://pbs.twimg.com/media/FQGLoR6akAIGbYI.jpg', 'img_file': '4.jpg'}, {'text': '見えちゃいまひた😆', 'img': 'http://pbs.twimg.com/media/FPgK0ScaAAQ2gsU.jpg', 'img_file': '5.jpg'}, {'text': 'みんないつもコメントありがとぉ〜🥰', 'img': 'http://pbs.twimg.com/media/FOSOdnWacAcyjpr.jpg', 'img_file': '6.jpg'}, {'text': '気づいたら4000フォロワーいってました🎉', 'img': 'http://pbs.twimg.com/media/FMqjzHKaIAA_0Yk.jpg', 'img_file': '7.jpg'}, {'text': 'お久しぶりです🌹', 'img': 'http://pbs.twimg.com/media/FMmLQ2JakAE3y8b.jpg', 'img_file': '8.jpg'}, {'text': 'この時ぐらいまで髪切ろうか迷ってる、、', 'img': 'http://pbs.twimg.com/media/FKLxVsRaIAEto1-.jpg', 'img_file': '9.jpg'}, {'text': '今夜は脚メインでいかせてください😳', 

Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}, {'text': 'このブラ盛れるっ😋✨', 'img': 'http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg', 'img_file': '3.jpg'}, {'text': 'おっぱいようでございます🤗', 'img': 'http://pbs.twimg.com/media/FQGLoR6akAIGbYI.jpg', 'img_file': '4.jpg'}, {'text': '見えちゃいまひた😆', 'img': 'http://pbs.twimg.com/media/FPgK0ScaAAQ2gsU.jpg', 'img_file': '5.jpg'}, {'text': 'みんないつもコメントありがとぉ〜🥰', 'img': 'http://pbs.twimg.com/media/FOSOdnWacAcyjpr.jpg', 'img_file': '6.jpg'}, {'text': '気づいたら4000フォロワーいってました🎉', 'img': 'http://pbs.twimg.com/media/FMqjzHKaIAA_0Yk.jpg', 'img_file': '7.jpg'}, {'text': 'お久しぶりです🌹', 'img': 'http://pbs.twimg.com/media/FMmLQ2JakAE3y8b.jpg', 'img_file': '8.jpg'}, {'text': 'この時ぐらいまで髪切ろうか迷ってる、、', 'img': 'http://pbs.twimg.com/media/FKLxVsRaIAEto1-.jpg', 'img_file': '9.jpg'}, {'text': '今夜は脚メインでいかせてください😳', 

Unexpected parameter: id
Unexpected parameter: include_entities
Unexpected parameter: lang


{'tweet': [{'text': '五月病予防にどうですか？🥴', 'img': 'http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg', 'img_file': '1.jpg'}, {'text': 'なんか知らない間に凄いことになってました😂', 'img': 'http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg', 'img_file': '2.jpg'}, {'text': 'このブラ盛れるっ😋✨', 'img': 'http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg', 'img_file': '3.jpg'}, {'text': 'おっぱいようでございます🤗', 'img': 'http://pbs.twimg.com/media/FQGLoR6akAIGbYI.jpg', 'img_file': '4.jpg'}, {'text': '見えちゃいまひた😆', 'img': 'http://pbs.twimg.com/media/FPgK0ScaAAQ2gsU.jpg', 'img_file': '5.jpg'}, {'text': 'みんないつもコメントありがとぉ〜🥰', 'img': 'http://pbs.twimg.com/media/FOSOdnWacAcyjpr.jpg', 'img_file': '6.jpg'}, {'text': '気づいたら4000フォロワーいってました🎉', 'img': 'http://pbs.twimg.com/media/FMqjzHKaIAA_0Yk.jpg', 'img_file': '7.jpg'}, {'text': 'お久しぶりです🌹', 'img': 'http://pbs.twimg.com/media/FMmLQ2JakAE3y8b.jpg', 'img_file': '8.jpg'}, {'text': 'この時ぐらいまで髪切ろうか迷ってる、、', 'img': 'http://pbs.twimg.com/media/FKLxVsRaIAEto1-.jpg', 'img_file': '9.jpg'}, {'text': '今夜は脚メインでいかせてください😳', 

In [58]:
load = json.load(open('erika_1837.json', 'r'))
l = load['tweet']


for i in l:
    print(i['img'])
    img_response = requests.get(i['img'])

http://pbs.twimg.com/media/FRoRVE7akAAchJJ.jpg
http://pbs.twimg.com/media/FQojlOfaMAUQu2s.jpg
http://pbs.twimg.com/media/FQg_3_CakAArWpn.jpg
http://pbs.twimg.com/media/FQGLoR6akAIGbYI.jpg
http://pbs.twimg.com/media/FPgK0ScaAAQ2gsU.jpg
http://pbs.twimg.com/media/FOSOdnWacAcyjpr.jpg
http://pbs.twimg.com/media/FMqjzHKaIAA_0Yk.jpg
http://pbs.twimg.com/media/FMmLQ2JakAE3y8b.jpg
http://pbs.twimg.com/media/FKLxVsRaIAEto1-.jpg
http://pbs.twimg.com/media/FJoY94faUAQfYvY.jpg
http://pbs.twimg.com/media/FIfkEUVaQAwkKjS.jpg
http://pbs.twimg.com/media/FH-Pwi8aMAAbtr_.jpg
http://pbs.twimg.com/media/FGQV6hIUYAUJ7Dg.jpg
http://pbs.twimg.com/media/FFS-_ksaIAAZQlK.jpg
http://pbs.twimg.com/media/FDJxZ73agAA_w6P.jpg
http://pbs.twimg.com/media/FB5T9boUUAALg39.jpg
http://pbs.twimg.com/media/FBBboFtVgAgK5NZ.jpg
http://pbs.twimg.com/media/FA2QahWVkAMcwan.jpg
http://pbs.twimg.com/media/FAYRcr4UcAQpA5e.jpg
http://pbs.twimg.com/media/FAH80_SVUAYEJf4.jpg
http://pbs.twimg.com/media/E_pcyTMUYAEzlmx.jpg
http://pbs.tw

NG エロアカウントからテキストだけもらう

In [1]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.count = 5
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []

        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力
        # テキストだけ
        target_tweets = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'])
        print(target_tweets)

        while True:
            next_token = target_tweets.meta['next_token']

            for tweets in target_tweets.data:
                text = tweets.data['text']
                tweet['tweet'].append({"text": text})

            with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)

            target_tweets = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'])

            if next_token is None:
                break

        # if next_token:
        #     while True:
        #         target_tweets_next = self.client.get_users_tweets(target_response.data['id'], max_results=100, exclude=['retweets', 'replies'], pagination_token=next_token)
        #         next_token = target_tweets_next.meta['next_token']

        #         for tweets in target_tweets_next.data:
        #             text = tweets.data['text']
        #             print(text)

        #             tweet['tweet'].append({"text": text})
        #             print(tweet)

        #             with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
        #                 json.dump(tweet, f, indent=4, ensure_ascii=False)

        #         if next_token is None:
        #             break

        # for tweets in tweepy.Paginator(self.client.get_users_tweets, target_response.data['id'], max_results=10, media_fields=['preview_image_url', 'url'], expansions=['attachments.media_keys'], tweet_fields=["entities"]).flatten(limit=250):


In [ ]:
t = Tweet_text('hikarinspecial')
t.text_get

# テキストだけ取得（paginatorを使う）

In [15]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.count = 5
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []
        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力

        for tweets in tweepy.Paginator(self.client.get_users_tweets,target_response.data['id'], exclude=['retweets', 'replies'],max_results=100).flatten(limit=1000):

            text = tweets.data['text']
            tweet['tweet'].append({"text": text})

            with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
                json.dump(tweet, f, indent=4, ensure_ascii=False)


In [18]:
t = Tweet_text('Lillian46612363')
t.text_get()

In [2]:
import tweepy
import API_config_katudon
import json


class Tweet_text:

    def __init__(self, username):

        self.client = tweepy.Client(consumer_key=API_config_katudon.API_KEY, consumer_secret=API_config_katudon.API_SECRET, access_token=API_config_katudon.ACCESS_TOKEN, access_token_secret=API_config_katudon.ACCESS_TOKEN_SECRET, bearer_token=API_config_katudon.Bearer_token, wait_on_rate_limit=True)
        self.__username = username

    def text_get(self):
        """_summary_
        usernameにアカウント名を入力
        """

        tweet: dict[dict] = {}
        tweet['tweet'] = []
        target_response = self.client.get_user(username=self.__username) # usernameにアカウント名を入力

        for tweets in tweepy.Paginator(self.client.get_users_tweets,target_response.data['id'], exclude=['replies'],max_results=100).flatten(limit=3000):

            self.client.delete_tweet(tweets.id)

            # JSON保存
            # text = tweets.data['text']
            # tweet['tweet'].append({"text": text})
            # with open(f'{self.__username}.json', 'w', encoding='utf-8') as f:
            #     json.dump(tweet, f, indent=4, ensure_ascii=False)

t = Tweet_text('katudon')
t.text_get()